<a href="https://colab.research.google.com/github/jinseriouspark/commerce-llm/blob/main/evaluation_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KeyLLM with OpenAI

- 모델 준비하기
- prompt 결과물 정리하기

In [4]:
!pip install openai
!pip install tiktoken
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 8.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [5]:
# 참고자료 : https://platform.openai.com/docs/tutorials/web-qa-embeddings
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')


In [6]:
import openai
from keybert.llm import OpenAI
from keybert import KeyLLM

# Create your LLM
client = openai.OpenAI(api_key=MY_API_KEY)
llm = OpenAI(client)

# Load it in KeyLLM
kw_model = KeyLLM(llm)

# Extract keywords
document = "The website mentions that it only takes a couple of days to deliver but I still have not received mine."
keywords = kw_model.extract_keywords(document)

ModuleNotFoundError: No module named 'keybert'

In [7]:


# create llm
with open('/content/drive/MyDrive/commerce-llm/env/openai_apikey.txt', 'r') as f:
  openai.api_key = f.readline().strip()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/commerce-llm/env/openai_apikey.txt'

In [ ]:
criteria_1_representativeness = """
Human: You will be given all reviews and the keywords of reviews.
Your task is to rate the keywords on criteria.
Please make sure you read and understand criteria and steps carefully. Please keep this document open while reviewing, and refer to it as needed.

<criteria>
Representativeness (1-5) - the extent to which the keywords captures key words across the entirety of the reviews, demonstrating a comprehensive understanding.
</criteria>

<steps>
Step 1. Carefully read the reviews and keywords provided.
Step 2. Evaluate how representative the keywords is across the entirety of the reviews, considering coherence.
Step 3. Rate the representativeness on a scale of 1 to 5:
    1 - Insignificantly Representative: The keywords fails to capture key words from the reviews, lacking a comprehensive understanding of the content.
    2 - Barely Representative: The keywords includes only a few relevant keywords, with significant gaps in understanding the reviews.
    3 - Moderately Representative: The keywords captures some key words, but misses others, resulting in a partial understanding of the reviews.
    4 - Largely Representative: The keywords effectively includes relevant keywords, demonstrating a good understanding of the reviews, although some areas may be lacking.
    5 - Fully Representative: The keywords comprehensively captures key words from the reviews, exhibiting a thorough understanding of the content.
Step 4. Provide the criteria rating from 1 to 5 based solely on the provided criteria.
Step 5. 4. Provide specific feedback on the issues you identified and suggestions for improvement. Be constructive and professional.
Step 6. Submit your criteria score and feedback comments to appropriate team member for consideration.
</steps>

<output_format>
The output is provided in json format and has `thoughts`, `criteria_rating`, and `feedback` as keys.
For detailed output format description, please check the provided json format.

{json_schema}
{'thoughs': 'your_thought_',
'criteria_rating' : 1,
'feedback' : 'your_feedback'
}
"""


In [ ]:
llm = TextGeneration( prompt = criteria_1_representativeness)
kw_model = KeyLLM(llm)

# KEYWORD EXTRACTION BY keybert
- keybert 란?
  - CountVectorizer 를 사용하여 문장 속 keyword, keyphrase  후보를 만든다
  - Language model을 사용해서 문장과 키워드를 각각 임베딩 한다 (sentence-transformers 등을 사용)
  - 인풋 문장과 후보 키워들 간의 코사인ㅇ 유사도를 계산하고, 가장 큰 유사도부터 '키워드' 라고 지정한다

In [ ]:
# KeyLLM with KeyBERT + Mistral 7b

In [ ]:
%%capture
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers
!pip install keybert ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers